<a href="https://colab.research.google.com/github/bmj0428/CSE6521/blob/main/TextFooler_241111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jind11/TextFooler.git

%cd TextFooler

In [ ]:
!pip install nltk boto3 pattern datasets


In [1]:
%cd TextFooler

/content/TextFooler


In [ ]:
%cd ESIM
!python setup.py install
%cd ..


In [ ]:
!pip install transformers==4.21.0
!pip install --upgrade tensorflow tensorflow_hub


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# PyTorch 기반의 BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# 모델과 토크나이저를 로컬 디렉토리에 저장 (safetensors 사용하지 않기)
model.save_pretrained('/content/models/bert_model', use_safetensors=False)
tokenizer.save_pretrained('/content/models/bert_model')


In [5]:
!mv /content/models/bert_model/config.json /content/models/bert_model/bert_config.json


In [6]:
!ls /content/models/bert_model/

bert_config.json  pytorch_model.bin  special_tokens_map.json  tokenizer_config.json  vocab.txt


In [ ]:
from datasets import load_dataset
import pandas as pd

# Hugging Face의 'yelp_polarity' 데이터셋을 로드합니다.
dataset = load_dataset('yelp_polarity', split='train')

# 데이터셋을 판다스 데이터프레임으로 변환
df = pd.DataFrame(dataset)

# 데이터프레임의 컬럼 확인
print(df.columns)  # ['text', 'label']

# 레이블과 텍스트 컬럼만 선택, 레이블을 먼저
df = df[['label', 'text']]

# 데이터프레임에서 레이블이 정수로 되어 있는지 확인 (레이블 컬럼이 정수형이어야 함)
df['label'] = df['label'].astype(int)

# 공백으로 구분된 텍스트 파일로 저장 (레이블 먼저, 텍스트 나중)
with open('/content/yelp_reviews_no_header.txt', 'w', encoding='utf8') as f:
    for _, row in df.iterrows():
        label = row['label']
        # 텍스트 내의 개행 문자를 제거하고 공백으로 대체
        text = row['text'].replace('\n', ' ').replace('\r', ' ').strip()
        # 레이블과 텍스트를 공백으로 구분하여 저장
        f.write(f"{label} {text}\n")

# 저장된 파일 일부 확인
with open('/content/yelp_reviews_no_header.txt', 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        if i < 5:  # 첫 5줄만 출력
            print(line)


In [8]:
!python comp_cos_sim_mat.py /content/counter-fitted-vectors.txt /content/counter-fitted-cos-sim.npy 10000

Embeddings shape: (10000, 300)
Cosine similarity matrix saved to /content/counter-fitted-cos-sim.npy


In [9]:
!ls /content/


counter-fitted-cos-sim.npy  models	 TextFooler
counter-fitted-vectors.txt  sample_data  yelp_reviews_no_header.txt


In [19]:
!python attack_classification.py \
  --dataset_path /content/yelp_reviews_no_header.txt \
  --target_model bert \
  --target_model_path /content/models/bert_model/ \
  --counter_fitting_embeddings_path /content/counter-fitted-vectors.txt \
  --USE_cache_path /content/USE_cache/ \
  --data_size 10

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-11-11 08:49:48.865258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 08:49:48.885738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 08:49:48.891767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 08:49:48.909521: I tensorflow/core/platform/cpu_feature